In [ ]:
! pip install -U ebonite flask flasgger==0.9.3 scikit-learn ebonite==0.5.2

In [1]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression

X, y = load_diabetes(True)

lr = LinearRegression()
lr.fit(X, y)

/Users/mike0sv/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass return_X_y=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


LinearRegression()

In [2]:
import ebonite

In [3]:
model = ebonite.create_model(lr, X, model_name='diabetes_model_1')

In [4]:
model

Model(id=None,name=diabetes_model_1)

In [5]:
from pprint import pprint

In [6]:
from pyjackson import serialize

pprint(serialize(model))

{'author': 'mike0sv',
 'creation_date': '2020-06-03 01:37:20.473379 ',
 'name': 'diabetes_model_1',
 'params': {'python_version': '3.6.7'},
 'requirements': {'requirements': [{'module': 'sklearn',
                                    'type': 'installable',
                                    'version': '0.23.1'},
                                   {'module': 'numpy',
                                    'type': 'installable',
                                    'version': '1.18.2'}]},
 'wrapper_meta': {'type': 'ebonite.ext.sklearn.model.SklearnModelWrapper'}}


In [7]:
ebnt = ebonite.Ebonite.local(clear=True)

In [8]:
task = ebnt.get_or_create_task('my_prj', 'diabetes_task')

In [9]:
task

Task(id=0,name=diabetes_task)

In [10]:
pprint(serialize(task))

{'author': 'mike0sv',
 'creation_date': '2020-06-03 01:40:28.271587 ',
 'id': 0,
 'name': 'diabetes_task',
 'project_id': 0}


In [11]:
task.push_model(model)

Model(id=0,name=diabetes_model_1)

In [12]:
pprint(serialize(model))

{'artifact': {'blobs': {'methods.json': {'path': '/Users/mike0sv/PycharmProjects/zyfra/ebonite/ebonite/examples/notebook_tutorial/.ebonite/artifacts/0/methods.json',
                                         'type': 'local_file'},
                        'model.pkl': {'path': '/Users/mike0sv/PycharmProjects/zyfra/ebonite/ebonite/examples/notebook_tutorial/.ebonite/artifacts/0/model.pkl',
                                      'type': 'local_file'},
                        'requirements.json': {'path': '/Users/mike0sv/PycharmProjects/zyfra/ebonite/ebonite/examples/notebook_tutorial/.ebonite/artifacts/0/requirements.json',
                                              'type': 'local_file'}},
              'type': 'blobs'},
 'author': 'mike0sv',
 'creation_date': '2020-06-03 01:37:20.473379 ',
 'id': 0,
 'name': 'diabetes_model_1',
 'params': {'python_version': '3.6.7'},
 'requirements': {'requirements': [{'module': 'sklearn',
                                    'type': 'installable',
     

In [13]:
! ls -la .ebonite/artifacts/0

total 24
drwxr-xr-x  5 mike0sv  staff  160 Jun  3 04:40 .
drwxr-xr-x  3 mike0sv  staff   96 Jun  3 04:40 ..
-rw-r--r--  1 mike0sv  staff  228 Jun  3 04:40 methods.json
-rw-r--r--  1 mike0sv  staff  754 Jun  3 04:40 model.pkl
-rw-r--r--  1 mike0sv  staff  150 Jun  3 04:40 requirements.json


In [14]:
model.id

0

In [15]:
del model

In [16]:
model = ebnt.get_model('diabetes_model_1', task)

In [17]:
model

Model(id=0,name=diabetes_model_1)

In [18]:
from ebonite.runtime import run_model_server

In [19]:
from ebonite.ext.flask.server import FlaskServer

In [20]:
run_model_server(model, FlaskServer())

2020-06-03 04:43:31,282 [INFO] ebonite_runtime: Starting Ebonite runtime with loader MockLoader and server FlaskServer ...
2020-06-03 04:43:31,287 [INFO] ebonite_runtime: Running server <ebonite.ext.flask.server.FlaskServer object at 0x1a2400cc88>
 * Serving Flask app "ebonite.ext.flask.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jun/2020 04:43:58] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [03/Jun/2020 04:43:58] "GET /apidocs HTTP/1.1" 308 -
127.0.0.1 - - [03/Jun/2020 04:43:58] "GET /apidocs/ HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 04:43:58] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 04:43:58] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 04:43:58] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 04:43:58] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 04:43:59] "GET /flasgger_static/favicon-32x32.png HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 04:43:59] "GET /apispec_1.json HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 04:44:29] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 04:44:41] "POST /predict HTTP/1.1" 400 -


In [21]:
image = ebnt.build_image('nb_example_diabetes', model, FlaskServer(), force_overwrite=True)

2020-06-03 04:46:20,791 [INFO] ebonite: Built image nb_example_diabetes:latest


In [23]:
instance = ebnt.create_instance('nb_example_diabetes', image, ports_mapping={9000:80})

In [24]:
instance.is_running()

False

In [25]:
instance.run()

In [26]:
! docker ps

CONTAINER ID        IMAGE                        COMMAND                  CREATED             STATUS              PORTS                  NAMES
55d620a9c3e3        nb_example_diabetes:latest   "/usr/bin/supervisord"   10 seconds ago      Up 10 seconds       0.0.0.0:80->9000/tcp   nb_example_diabetes


In [27]:
instance.is_running()

True

In [28]:
from ebonite.ext.flask.client import HTTPClient

In [29]:
client = HTTPClient('localhost', 80)

In [30]:
client.methods

{'predict': Method(name='predict', args=[Argument(name='vector', type=<class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(None, 10),dtype=float64]'>)], out_type=<class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(None,),dtype=float64]'>)}

In [31]:
import numpy as np
client.predict(np.array([[1.,2,3,4,5,6,7,8,9,10]]))

array([10980.71155335])

In [33]:
def is_bad(data):
    preds = lr.predict(data)
    return preds > 50

In [34]:
model2 = ebonite.create_model(is_bad, X, 'diabetes_model_2')
task.push_model(model2)

Model(id=1,name=diabetes_model_2)

In [35]:
pprint(serialize(model2))

{'artifact': {'blobs': {'methods.json': {'path': '/Users/mike0sv/PycharmProjects/zyfra/ebonite/ebonite/examples/notebook_tutorial/.ebonite/artifacts/1/methods.json',
                                         'type': 'local_file'},
                        'model.pkl': {'path': '/Users/mike0sv/PycharmProjects/zyfra/ebonite/ebonite/examples/notebook_tutorial/.ebonite/artifacts/1/model.pkl',
                                      'type': 'local_file'},
                        'requirements.json': {'path': '/Users/mike0sv/PycharmProjects/zyfra/ebonite/ebonite/examples/notebook_tutorial/.ebonite/artifacts/1/requirements.json',
                                              'type': 'local_file'}},
              'type': 'blobs'},
 'author': 'mike0sv',
 'creation_date': '2020-06-03 01:53:55.378329 ',
 'id': 1,
 'name': 'diabetes_model_2',
 'params': {'python_version': '3.6.7'},
 'requirements': {'requirements': [{'module': 'numpy',
                                    'type': 'installable',
       

In [36]:
image2 = ebnt.build_image('nb_example_diabetes2', model2, FlaskServer(), force_overwrite=True)

2020-06-03 04:55:10,571 [INFO] ebonite: Built image nb_example_diabetes2:latest


In [37]:
instance2 = ebnt.create_instance('nb_example_diabetes2', image2, ports_mapping={9000:81})

In [38]:
instance2.run()

In [39]:
client2 = HTTPClient('localhost', 81)

In [40]:
client2.predict(np.array([[1.,2,3,4,5,6,7,8,9,10]]))

array([ True])

In [41]:
client2.methods

{'predict': Method(name='predict', args=[Argument(name='vector', type=<class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(None, 10),dtype=float64]'>)], out_type=<class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(None,),dtype=bool]'>)}